In [ ]:
import networkx as nx
import numpy as np
#import seaborn as sb
#import scipy
import matplotlib.pyplot as plt
import cvxpy as cp

# Matrix Generation

### Utility Functions

In [ ]:
sample = np.random.sample

In [ ]:
#Given a list of weights, will pick a random index with likelihood corresponding to the weight
def pickRandomIndex(weights,total):
    t = sample() * total;
    i = 0;
    for w in weights:
        t -= w
        if t < 0:
            return i;
        i += 1;
    raise ValueError('total is bigger than the sum of weights')

### Generation Functions

In [ ]:
#random matrix with edge density p
def randomPMatrix(n,p):
    A = np.zeros([n,n]);
    for i in range(1,n):
        for j in range(i):
            if sample() < p:
                A[i,j] = 1;
                A[j,i] = 1;
    return A

In [ ]:
#random d-regular graph
errorbound = 10;

def randomDRegular(n,d):
    A = np.ones([1,1]) * -1
    #num = 1
    while np.all(A == -1):
        A = tryRandomDRegular(n,d)
        #print("n = ",n, "\n Attempt:",num)
        #num += 1
    return A


def tryRandomDRegular(n,d):
    A = np.zeros([n,n]);
    weights = d*np.ones([n]);
    total = d*n
    error = 0;
    while total > 0:
        i = pickRandomIndex(weights, total)
        j = pickRandomIndex(weights, total)
        if i != j and A[i,j] == 0:
            A[i,j] = 1
            A[j,i] = 1
            weights[i] -= 1
            weights[j] -= 1
            total -= 2
            error = 0
        else:
            error += 1
        if error >= errorbound:
            return np.ones([1,1])*-1
    return A

In [ ]:
#random Barabási–Albert small world graph
#Note that every vertex we add will add m connections and thus increase total degree by 2m
#Therefore for n -> \infinity we see that the average degree will approach 2m

def BASmallWorld(n,m):
    A = np.zeros([n,n]);
    weights = np.zeros([n]);
    for i in range(m):
        weights[i] = 1
        A[i,m] = 1
        A[m,i] = 1
    weights[m] = m
    total = 2*m
    for i in range(m + 1,n):
        d = m
        while d > 0:
            j = pickRandomIndex(weights,total)
            if i != j and A[i,j] == 0:
                A[i,j] = 1
                A[j,i] = 1
                weights[i] += 1
                weights[j] += 1
                total += 1
                d -= 1
        total += m
    return A

In [ ]:
#random Watts–Strogatz small world graph
#Note: input K/2 instead of K
def WSSmallWorld(n,K, b):
    A = np.zeros([n,n]);
    weights = np.zeros([n]);
    for i in range(n):
        for k in range(K):
            j = np.mod(i + 1 + k,n)
            A[i,j] = 1
            A[j,i] = 1
    for i in range(n):
        for k in range(K):
            #chance = beta to rewire
            if sample() < b:

                j = np.mod(i + 1 + k, n)
                l = j
            
                #find l that i is not connected to
                while A[i,l] == 1 or i == l:
                    l = np.random.randint(n)
                
                #rewire to l
                A[i,j] = 0
                A[j,i] = 0
                A[i,l] = 1
                A[l,i] = 1
    return A

### Matrix Preperation

In [ ]:
#How does the average Degree depend on n?
def averageDegree(n):
    return 10  #Currently keeping the degree constant as n increases in size

In [ ]:
#Generate matrices for different sizes
matrixSizes = np.arange(25,200,25)

pMatrices = []
dRegularMatrices = []
BAMatrices = []
WSMatrices = []
rewireChance = .3

for i in range(matrixSizes.shape[0]):
    n = matrixSizes[i]
    k = averageDegree(n)
    
    pMatrices.append(randomPMatrix(n, k/n))
    dRegularMatrices.append( randomDRegular(n,k))
    BAMatrices.append( BASmallWorld(n, np.floor(k/2).astype(int)))
    WSMatrices.append( WSSmallWorld(n, np.floor(k/2).astype(int), rewireChance))

In [ ]:
#Run individual lists/entries to view the results
WSMatrices

# Methods

## Standard Methods

In [ ]:
#supremum lower
def sup_lower_approx(VV, n):
    return int(np.ceil(1/np.max(VV)))

In [ ]:
#greedy lower
def lower_approx(VV, n):
    vflat = VV.reshape(n*n)
    res = 0
    count = 0
    sorted = np.argsort(vflat)
    while res < 1:
        count += 1
        res += vflat[sorted[-count]]
    return count

In [ ]:
#linear lower
def linear_approx(V2, n):
    x = cp.Variable(n)
    y = cp.Variable(n)
    u = cp.Variable((n,n))

    c1 = cp.sum(cp.multiply(V2, u))
    cx = cp.reshape(x,(n,1)) @ cp.reshape(np.ones([1,n]),(1,n))
    cy = cp.reshape(np.ones([1,n]),(n,1)) @ cp.reshape(y,(1,n))

    constraints = [cp.vec(u) >= 0, cp.vec(u) <= 1, 
                   0 <= x, x <= 1, 0 <= y, y <= 1, 
                   x >= rx, x <= rx + 1, y >= ry, y <= ry + 1,
                   c1 >= 1, u <= cx, u <= cy, cx + cy - 1  <= u
                   ]
    prob = cp.Problem(cp.Minimize(cp.sum(u)),constraints)
    sol = prob.solve()
    print("x = ", x.value)
    print("y = ", y.value)
    print("u = ", u.value)
    print("sol = ", np.ceil(sol))
    return int(np.ceil(sol))

In [ ]:
#sdp lower
def sdp_approx(VV, n):
    x = cp.Variable(2*n)
    u = cp.Variable((2*n,2*n))
    
    Z = np.zeros((n,n))
    O = np.ones((n,n))
    Vsym = np.concatenate((np.concatenate((Z,0.5*VV), axis =1), np.concatenate((0.5*VV.T, Z), axis =1)))
    Idsym = np.concatenate((np.concatenate((Z,0.5*O), axis =1), np.concatenate((0.5*O, Z), axis =1)))

    c1 = cp.sum(cp.multiply(Vsym, u))
    cx = cp.reshape(x,(2*n,1)) @ cp.reshape(np.ones([1,2*n]),(1,2*n))
    cy = cp.reshape(np.ones([1,2*n]),(2*n,1)) @ cp.reshape(x,(1,2*n))
    ctrace = cp.trace(u - cp.diag(x))
    
    A11= cp.reshape(1, (1, 1))
    A12= cp.reshape(x, (1, 2*n))
    A1 = cp.hstack([A11, A12])

    A21= cp.reshape(x, (2*n,1))
    A22= u
    A2 = cp.hstack([A21, A22])
    M = cp.vstack([A1, A2])
    

    constraints = [cp.vec(u) >= 0, cp.vec(u) <= 1, 0 <= x, x <= 1, c1 >= 1, u <= cx, u <= cy, cx + cy - 1  <= u, ctrace == 0, M >> 0]
    prob = cp.Problem(cp.Minimize(cp.sum(cp.multiply(Idsym, u))), constraints)
    return int(np.ceil(prob.solve()))

In [ ]:
#k-width upper
def upper_bound_2(VV, n):
    res = []
    for i in range(n):
        for j in range(i+1,n):
            ind_subset = [i,j]
            VV_tr = VV[ind_subset, :]
            sums = np.sum(VV_tr, axis = 0)
            argsorted_sums = np.argsort(sums)
            cumsum = 0
            count = 0
            while cumsum < 1:
                count += 1
                cumsum += sums[argsorted_sums[-count]]
            res.append(count*2)
            VV_tr = VV[:,ind_subset]
            sums = np.sum(VV_tr, axis = 1)
            argsorted_sums = np.argsort(sums)
            cumsum = 0
            count = 0
            while cumsum < 1:
                count += 1
                cumsum += sums[argsorted_sums[-count]]
            res.append(count*2)
    return np.min(res)

## Custom Methods

In [ ]:
#custom linear
def linear_mod(V2, n, rx, ry):
    x = cp.Variable(n)
    y = cp.Variable(n)
    u = cp.Variable((n,n))

    c1 = cp.sum(cp.multiply(V2, u))
    cx = cp.reshape(x,(n,1)) @ cp.reshape(np.ones([1,n]),(1,n))
    cy = cp.reshape(np.ones([1,n]),(n,1)) @ cp.reshape(y,(1,n))

    constraints = [cp.vec(u) >= 0, cp.vec(u) <= 1, 
                   0 <= x, x <= 1, 0 <= y, y <= 1, 
                   x >= rx, x <= rx + 1, y >= ry, y <= ry + 1,
                   c1 >= 1, u <= cx, u <= cy, cx + cy - 1  <= u
                   ]
    prob = cp.Problem(cp.Minimize(cp.sum(u)),constraints)
    sol = prob.solve()
    print("x = ", x.value)
    print("y = ", y.value)
    print("u = ", u.value)
    return int(np.ceil(sol))

In [ ]:
#custom greedy
def no_gaps_mod(VV, n):
    P = sortsummatrix(VV)
    res = []
    for ki in range(n):
        for kj in range(ki+1,n):
            #sum up each row restricted to ki, kj
            ind_subset = [ki,kj]
            VV_tr = VV[:,ind_subset]
            sums = np.sum(VV_tr, axis = 1)
            
            for i in range (2,n):
                #Add the restricted sum, to the best remaining i - 2 elements in the row
                T = sums + P[:, i - 2]
                T = np.sort(T)

                #keep selecting the best rows until we exceed 1
                cumsum = 0
                count = 0
                while cumsum < 1:
                    cumsum += T[count]
                    count += 1
                res.append(i*count)
    return np.min(res)

In [ ]:
def sortsum(c):
    a = np.sort(c)
    return np.cumsum(a) - a

def sortsummatrix(VV):
    n = VV.shape[0]
    A = np.zeros(VV.shape);
    for i in range(n):
        A[i,:] = sortsum(VV[i,:])
    return A

In [ ]:
M = np.array([[3,2,3],[2,2,1]])
N = M[:,[0,2]]
print(sortsummatrix(M)[:,2])
sums = np.sum(N,axis=1)
print(N)
print(sums)

In [ ]:
n = 10
M = np.ones([n,n])/11
no_gaps_mod(M,n)

# Experiments

# Results